In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=204c2548efc67d1f10044dd5a2a0e1a61ef13c264ab59f36cdce4e8187df7b1b
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 19.66 ,random_state=0 , l1_ratio= 0.997, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_9_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=853498, Mon Apr 13 13:03:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 1.058281675585931e+06 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 1.055140590945697e+06 1.1e+00 1.21e-01  1e-01  1e-01 0:00.0
    3     39 1.041600376260075e+06 1.1e+00 1.25e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1058281.67558593] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1057717.33736941] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -2.971428564255999e+05 1.6e+00 5.97e-03  5e-03  6e-03 0:01.0
  200   2600 -2.987667767998937e+05 1.9e+00 1.17e-04  8e-05  1e-04 0:01.9
  300   3900 -2.987673192609453e+05 2.2e+00 1.99e-06  1e-06  2e-06 0:02.8
  400   5200 -2.987673193822771e+05 2.2e+00 3.27e-08  2e-08  2e-08 0:03.7
  500   6500 -2.987673193822897e+05 2.4e+00 2.44e-09  1e-09  1e-09 0:04.7
  600   7800 -2.987673193822899e+05 3.2e+00 1.57e-09  6e-10  1e-09 0:05.6
  700   9100 -2.987673193822897e+05 3.9e+00 1.71e-09  7e-10  1e-09 0:06.6
  800  10400 -2.987673193822897e+05 4.5e+00 1.38e-09  5e-10  9e-10 0:07.6
  900  11700 -2.987673193822894e+05 5.9e+00 7.53e-10  2e-10  5e-10 0:08.5
 1000  13000 -2.987673193822899e+05 7.5e+00 4.53e-10  1e-10  3e-10 0:09.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 -2.987673193822897e+05 9.2e+00 6.19e-10  2e-10  5e-10 0:10.4
 1200  15600 -2.987673193822899e+05 1.2e+01 6.44e-10  2e-10  5e-10 0:11.4
 1300  16900 -2.987673193822899e+05 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1012718.46714756] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1012736.76246987] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 9.999838620198685e+05 1.7e+00 4.65e-03  4e-03  4e-03 0:00.8
  200   2600 9.999777748923016e+05 1.8e+00 7.24e-05  5e-05  6e-05 0:01.6
  300   3900 9.999777735591113e+05 1.9e+00 9.69e-07  6e-07  7e-07 0:02.4
  400   5200 9.999777735589464e+05 2.0e+00 1.80e-08  8e-09  1e-08 0:03.2


...

  100   1300 9.999982473574028e+05 1.7e+00 6.40e-03  5e-03  6e-03 0:00.8
  200   2600 9.999777749518041e+05 1.8e+00 8.19e-05  6e-05  7e-05 0:01.6
  300   3900 9.999777735590888e+05 2.0e+00 1.09e-06  6e-07  8e-07 0:02.3
  400   5200 9.999777735589464e+05 2.1e+00 1.91e-08  9e-09  1e-08 0:03.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=2.02e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=388)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=1.86e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=392)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=1.78e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=404)
  ')')
/usr/local/lib/pyth

  449   5837 9.999777735589464e+05 2.4e+00 1.29e-08  6e-09  9e-09 0:03.5
termination on tolfunhist=1e-12 (Mon Apr 13 13:15:09 2020)
termination on flat fitness=please (re)consider how to compute the fitness more elaborately if sigma=1.29e-08 is large (Mon Apr 13 13:15:09 2020)
final/bestever f-value = 9.999778e+05 9.999778e+05
incumbent solution: [-0.14705225  0.0379022  -0.08016317 -0.05231233 -0.10292836  0.00277484
  0.15697365  0.05729451 ...]
std deviations: [7.60379493e-09 6.53218706e-09 6.70514968e-09 7.32109645e-09
 5.94587614e-09 7.82898180e-09 7.67058454e-09 7.20104357e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=899761, Mon Apr 13 13:15:09 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 1.129581243651610e+06 1.0e+00 1.20e-01  1e-01  1e-01 0:00.0
    2     26 1.118067617972429e+06 1.1e+00 1.17e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=1.31e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=445)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=1.35e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=446)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=999977.773559, sigma=1.32e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=448)
  ')')
/usr/local/lib/pyth

    3     39 1.086939938573228e+06 1.1e+00 1.17e-01  1e-01  1e-01 0:00.0
   80   1040 -6.788123629558993e+10 2.5e+00 1.16e+02  1e+02  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:15:10 2020)
final/bestever f-value = -7.007104e+10 -7.007104e+10
incumbent solution: [-107.70551823  -27.20452023  -45.56997298  687.33525878   57.98866704
 -228.48333285  -78.1390298  -374.70328833 ...]
std deviations: [115.05722669 101.56433219 102.45074761 119.44686193 105.08684614
 106.1775577  108.35462797 115.26466277 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F9_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:25])
    SVM_Fun [i] = F9(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F9(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-146.98561449]), count=array([1])) 973.8446875692981
SVM
ModeResult(mode=array([-141.13506958]), count=array([1])) 5.375338815458509e-06
ELN
ModeResult(mode=array([1.01304355e+14]), count=array([1])) 269116438350239.44
